In [11]:
import os
import glob
import numpy as np

def convert_seg_to_bbox(seg_label_path, bbox_label_path):
    with open(seg_label_path, 'r') as f:
        lines = f.readlines()
    
    bbox_lines = []
    for line in lines:
        parts = line.strip().split()
        class_id = 0
        
        # Extract all x,y coordinates
        coords = [float(coord) for coord in parts[1:]]
        points = np.array(coords).reshape(-1, 2)
        
        # Find min/max to create bounding box
        x_min, y_min = points.min(axis=0)
        x_max, y_max = points.max(axis=0)
        
        # Convert to YOLO format (x_center, y_center, width, height)
        x_center = (x_min + x_max) / 2
        y_center = (y_min + y_max) / 2
        width = x_max - x_min
        height = y_max - y_min
        
        bbox_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    
    # Write to new label file
    with open(bbox_label_path, 'w') as f:
        f.write('\n'.join(bbox_lines))

# Process all segmentation labels
seg_label_dir = "../data/processed/meatballs/gt_masks"
bbox_label_dir = "../data/processed/meatballs_yolo/labels"
os.makedirs(bbox_label_dir, exist_ok=True)

for seg_file in glob.glob(os.path.join(seg_label_dir, "*.txt")):
    filename = os.path.basename(seg_file)
    bbox_file = os.path.join(bbox_label_dir, filename)
    convert_seg_to_bbox(seg_file, bbox_file)

In [ ]:
import shutil

# Copy the images (you don't need to change these)
src_img_dir = "../data/processed/meatballs/images"
dst_img_dir = "../data/processed/meatballs_yolo/images"
os.makedirs(dst_img_dir, exist_ok=True)

for img_file in glob.glob(os.path.join(src_img_dir, "*.png")):
    shutil.copy(img_file, dst_img_dir)


In [ ]:
import os
import random
import shutil
from pathlib import Path

def reorganize_existing_dataset(dataset_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, seed=42):
    """
    Reorganize an existing dataset with flat images and labels folders into train/val/test splits
    """
    # Set random seed for reproducibility
    random.seed(seed)
    
    # Input directories
    img_dir = os.path.join(dataset_dir, "images")
    label_dir = os.path.join(dataset_dir, "labels")
    
    # Get all image files before creating new directories
    image_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(image_files)
    
    # Create subdirectories
    for split in ["train", "val", "test"]:
        os.makedirs(os.path.join(img_dir, split), exist_ok=True)
        os.makedirs(os.path.join(label_dir, split), exist_ok=True)
    
    # Calculate split indices
    n_samples = len(image_files)
    train_end = int(n_samples * train_ratio)
    val_end = train_end + int(n_samples * val_ratio)
    
    # Split files
    train_files = image_files[:train_end]
    val_files = image_files[train_end:val_end]
    test_files = image_files[val_end:]
    
    # Move files to respective directories
    splits = {
        "train": train_files,
        "val": val_files,
        "test": test_files
    }
    
    for split, files in splits.items():
        for filename in files:
            # Move image
            src_img = os.path.join(img_dir, filename)
            dst_img = os.path.join(img_dir, split, filename)
            shutil.move(src_img, dst_img)
            
            # Move label (if exists)
            base_name = os.path.splitext(filename)[0]
            label_filename = f"{base_name}.txt"
            src_label = os.path.join(label_dir, label_filename)
            dst_label = os.path.join(label_dir, split, label_filename)
            
            if os.path.exists(src_label):
                shutil.move(src_label, dst_label)
    
    # Print statistics
    print(f"Dataset reorganization complete:")
    print(f"  Train: {len(train_files)} images")
    print(f"  Validation: {len(val_files)} images")
    print(f"  Test: {len(test_files)} images")
    
    # Create YAML file
    yaml_path = os.path.join(dataset_dir, "data.yaml")
    
    # Try to infer class names
    class_names = ["class0", "class1"]  # Default placeholder
    # Check for any label file to infer classes
    sample_dirs = [os.path.join(label_dir, split) for split in ["train", "val", "test"]]
    for dir_path in sample_dirs:
        label_files = [f for f in os.listdir(dir_path) if f.endswith('.txt')]
        if label_files:
            # Take the first label file and find max class ID
            with open(os.path.join(dir_path, label_files[0]), 'r') as f:
                class_ids = set()
                for line in f:
                    parts = line.strip().split()
                    if parts:
                        class_ids.add(int(parts[0]))
            
            if class_ids:
                num_classes = max(class_ids) + 1
                class_names = [f"class{i}" for i in range(num_classes)]
            break
    
    yaml_content = f"""
# YOLOv8 dataset config
path: {os.path.abspath(dataset_dir)}
train: images/train
val: images/val
test: images/test

# Classes
nc: {len(class_names)}
names: {class_names}
"""

    with open(yaml_path, 'w') as f:
        f.write(yaml_content)
    
    print(f"Dataset YAML created at: {yaml_path}")
    print("Note: Update the class names in the YAML file to match your actual classes.")
    
    return yaml_path

# Example usage
dataset_dir = "../data/processed/meatballs_yolo"  # Contains images/ and labels/ folders
yaml_file = reorganize_existing_dataset(dataset_dir)

Dataset reorganization complete:
  Train: 140 images
  Validation: 40 images
  Test: 20 images
Dataset YAML created at: ../data/processed/meatballs_yolo\dataset.yaml
Note: Update the class names in the YAML file to match your actual classes.


# let's train


In [14]:
from ultralytics import YOLO
import numpy as np
import torch

# Load your model
model = YOLO("yolov8n.pt")

def mc_inference_precise(image_path, num_samples=30):
    # Store original state
    training_state = model.model.training
    
    # Set model to eval mode (keeps batch norm using running stats)
    model.model.eval()
    
    # Manually set only dropout layers to train mode
    for module in model.model.modules():
        if isinstance(module, torch.nn.Dropout):
            module.training = True
    
    # Run inference
    results_list = []
    with torch.no_grad():
        for _ in range(num_samples):
            results = model(image_path)
            detections = results[0].boxes.data
            results_list.append(detections.cpu().numpy())
    
    # Restore original state
    model.model.train(training_state)
    
    return results_list

In [15]:
data_path = '../data/processed/meatballs_yolo/data.yaml'

results = model.train(
    data=data_path, 
    epochs=100, 
    imgsz=640,
    batch=16,
    patience=50,
    pretrained=True,
    device=0  # GPU id (0, 1, 2, etc.) or 'cpu'
)

Ultralytics 8.3.111  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=../data/processed/meatballs_yolo/data.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

train: Scanning C:\Users\gtoft\OneDrive\DTU\4_Semester_AS\Master_Thesis\auto-annotation-sam\data\processed\meatballs_yolo\labels\train... 140 images, 0 backgrounds, 0 corrupt: 100%|██████████| 140/140 [00:00<00:00, 883.39it/s]

train: New cache created: C:\Users\gtoft\OneDrive\DTU\4_Semester_AS\Master_Thesis\auto-annotation-sam\data\processed\meatballs_yolo\labels\train.cache


val: Fast image access  (ping: 0.00.0 ms, read: 878.9193.6 MB/s, size: 128.6 KB)


val: Scanning C:\Users\gtoft\OneDrive\DTU\4_Semester_AS\Master_Thesis\auto-annotation-sam\data\processed\meatballs_yolo\labels\val... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<00:00, 648.69it/s]

...

    100/100      2.92G     0.8714     0.3805     0.8744        611        640: 100%|██████████| 9/9 [00:01<00:00,  5.14it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.32it/s]

                   all         40       2118      0.993      0.999      0.995       0.75



100 epochs completed in 0.094 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 6.3MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics 8.3.111  Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.22s/it]


                   all         40       2118      0.993      0.998      0.995      0.753
Speed: 0.3ms preprocess, 4.0ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs\detect\train3


# Let's do inference

In [25]:
from ultralytics import YOLO
import cv2
import os
from IPython.display import display, HTML, Video

# Load your trained model
model_path = "runs/detect/train3/weights/best.pt"  # Update this to your model path
model = YOLO(model_path)

# Path to your video
video_path = "../data/raw/NewMeatballs2.mp4"  # Update this to your video path

# Output settings
output_folder = "yolo_inference"
os.makedirs(output_folder, exist_ok=True)
video_name = os.path.basename(video_path)
video_name_without_ext = os.path.splitext(video_name)[0]
output_path = os.path.join(output_folder, f"{video_name_without_ext}_pred.mp4")

# Run inference with tracking
results = model.track(
    source=video_path,
    conf=0.5,  # Confidence threshold
    iou=0.45,
    save=True,
    project=output_folder,
    name=video_name_without_ext,
    stream=False,  # Set to True for very large videos to process frame by frame
    persist=True,
)

# Print detection summary
print(f"Processing complete! Video saved to: {output_path}")



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/960) c:\Users\gtoft\OneDrive\DTU\4_Semester_AS\Master_Thesis\auto-annotation-sam\notebooks\..\data\raw\NewMeatballs2.mp4: 640x640 1 class0, 10.1ms
video 1/1 (frame 2/960) c:\Users\gtoft\OneDrive\DTU\4_Semester_AS\Master_Thesis\auto-annotation-sam\notebooks\..\data\raw\NewMeatballs2.mp4: 640x640 1 class0, 7.5ms
video 1/1 (frame 3/960) c:\Users\gtoft\OneDrive\DTU\4_Semester_AS\Master_Thesis\auto-annotation-sam\notebooks\..\data\raw\NewMe